![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and `ibm/granite-8b-code-instruct` to generate code based on instruction

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate support for code generating in watsonx. It introduces commands for defining prompt and model testing.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goal

The goal of this notebook is to demonstrate how to generate code using `ibm/granite-8b-code-instruct` watsonx model based on instruction provided by the user.


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Foundation Models on watsonx](#models)
- [Generate code based on instruction](#generate)
- [Generated code testing](#testing)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watsonxai-runtime" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).


### Install dependencies

In [1]:
%pip install -U ibm-watsonx-ai | tail -n 1

### Defining the watsonx.ai credentials
This cell defines the watsonx.ai credentials required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">documentation</a>.

In [2]:
import getpass

from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Please enter your watsonx.ai api key (hit enter): "),
)

### Defining the project ID
The Foundation Model requires project ID that provides the context for the call. We will obtain the ID from the project in which this notebook runs. Otherwise, please provide the project ID.

In [3]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

### API Client initialization

In [4]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials, project_id=project_id)

<a id="models"></a>
## Foundation Models on `watsonx.ai`

#### List available models

In [5]:
client.foundation_models.TextModels.show()

{'GRANITE_3_2_8B_INSTRUCT': 'ibm/granite-3-2-8b-instruct', 'GRANITE_3_2B_INSTRUCT': 'ibm/granite-3-2b-instruct', 'GRANITE_3_3_8B_INSTRUCT': 'ibm/granite-3-3-8b-instruct', 'GRANITE_3_8B_INSTRUCT': 'ibm/granite-3-8b-instruct', 'GRANITE_4_H_SMALL': 'ibm/granite-4-h-small', 'GRANITE_8B_CODE_INSTRUCT': 'ibm/granite-8b-code-instruct', 'GRANITE_GUARDIAN_3_8B': 'ibm/granite-guardian-3-8b', 'GRANITE_VISION_3_2_2B': 'ibm/granite-vision-3-2-2b', 'LLAMA_3_2_11B_VISION_INSTRUCT': 'meta-llama/llama-3-2-11b-vision-instruct', 'LLAMA_3_2_90B_VISION_INSTRUCT': 'meta-llama/llama-3-2-90b-vision-instruct', 'LLAMA_3_3_70B_INSTRUCT': 'meta-llama/llama-3-3-70b-instruct', 'LLAMA_3_405B_INSTRUCT': 'meta-llama/llama-3-405b-instruct', 'LLAMA_4_MAVERICK_17B_128E_INSTRUCT_FP8': 'meta-llama/llama-4-maverick-17b-128e-instruct-fp8', 'LLAMA_GUARD_3_11B_VISION': 'meta-llama/llama-guard-3-11b-vision', 'MISTRAL_MEDIUM_2505': 'mistralai/mistral-medium-2505', 'MISTRAL_SMALL_3_1_24B_INSTRUCT_2503': 'mistralai/mistral-small-3

You need to specify `model_id` that will be used for inferencing:

In [6]:
model_id = client.foundation_models.TextModels.GRANITE_8B_CODE_INSTRUCT

### Defining the model parameters

You might need to adjust model `parameters` for different models or tasks, to do so please refer to <a href="https://ibm.github.io/watsonx-ai-python-sdk/fm_model.html#metanames.GenTextParamsMetaNames" target="_blank" rel="noopener no referrer">documentation</a>.

In [7]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.TEMPERATURE: 0,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.STOP_SEQUENCES: ["<end·of·code>"],
}

### Initialize the model
Initialize the `ModelInference` class with previous set params.

In [8]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
    model_id=model_id, params=parameters, credentials=credentials, project_id=project_id
)

### Model's details

In [9]:
import json

print(json.dumps(model.get_details(), indent=2))

{
  "model_id": "ibm/granite-8b-code-instruct",
  "label": "granite-8b-code-instruct",
  "provider": "IBM",
  "source": "IBM",
  "indemnity": "IBM_COVERED",
  "functions": [
    {
      "id": "text_generation"
    }
  ],
  "short_description": "The Granite model series is a family of IBM-trained, dense decoder-only models, which are particularly well-suited for generative tasks.",
  "long_description": "Granite models are designed to be used for a wide range of generative and non-generative tasks with appropriate prompt engineering. They employ a GPT-style decoder-only architecture, with additional innovations from IBM Research and the open community.",
  "terms_url": "https://www.ibm.com/support/customer/csol/terms/?id=i126-6883",
  "input_tier": "class_1",
  "output_tier": "class_1",
  "number_params": "8b",
  "min_shot_size": 1,
  "task_ids": [
    "question_answering",
    "summarization",
    "classification",
    "generation",
    "code",
    "extraction",
    "code-generation",


<a id="generate"></a>
## Generate code based on instruction

Define instructions for the model with at-least one example.

In [10]:
instruction = """Using the directions below, generate Python code for the given task.

Input:
# Write a Python function that prints 'Hello World!' string 'n' times.

Output:
def print_n_times(n):
    for i in range(n):
        print("Hello World!")

<end of code>
"""

Prepare question for the model.

In [11]:
question = """Input:
# Write a Python function, which generates sequence of prime numbers.
# The function 'primes' will take the argument 'n', an int. It will return a list which contains all primes less than 'n'."""

### Generate the code using `ibm/granite-8b-code-instruct` model

Inter the model to generate the code, according to provided instruction.

In [12]:
result = model.generate_text(" ".join([instruction, question]))

Formatting the text to get the function itself

In [13]:
code_as_text = result.split("Output:")[1].split("<end of code>")[0]

<a id="testing"></a>
## Generated code testing

The resulting code looks as below.

In [14]:
print(code_as_text)


def primes(n):
    primes = []
    for num in range(2, n):
        is_prime = True
        for i in range(2, num):
            if num % i == 0:
                is_prime = False
                break
        if is_prime:
            primes.append(num)
    return primes




Use generated code to make it as function.

**Note**: Before executing this line, make sure the model's output visible above doesn't contain any malicious instructions.

In [15]:
exec(code_as_text)

Define the number 'n' for which the primes() function should process prime numbers.

In [16]:
n = 25

Test and run the generated function.

In [17]:
primes(n)

[2, 3, 5, 7, 11, 13, 17, 19, 23]

<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!
 
 You learned how to generate code based on instuction with `codellama-34b-instruct-hf` on watsonx. 
 
 Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors: 

 **Mateusz Szewczyk**, Software Engineer at watsonx.ai.

Copyright © 2024-2026 IBM. This notebook and its source code are released under the terms of the MIT License.